# Exception- und Event Handling mit Powershell <img src="https://cdn.shortpixel.ai/spai/w_149+q_lossy+ret_img+to_webp/https://www.itrust.ch/wp-content/uploads/2019/07/itrust-logo.png" align="right"/>

## Einführung

TODO

### Ziele
- Einführung in die Grundsätze des Exception Handlings
- Exceptions können in die grundlegenden Fehlerarten eingeteilt und passend behandelt werden
- Die Unterschiede und Gemeinsamkeiten zwischen Exceptions und Non-Terminating Errors sind bekannt



> NOTE: Weiterführende Informationen zum Thema Exception Handling inkl. Beispielen, können über den Microsoft Docs Artikel [Everything you wanted to know about exceptions](https://docs.microsoft.com/en-us/powershell/scripting/learn/deep-dives/everything-about-exceptions?view=powershell-7.1) bezogen werden.

### Import und Initialisieren von Modulen für Demo

In [68]:
Import-Module .\ExceptionHandlingHelper.psm1
$InformationPreference = "Continue"

Import-Module: The specified module '.\ExceptionHandlingHelper.psm1' was not loaded because no valid module file was found in any module directory.


## Was muss beim Exceptionhandling beachtet werden



## Grundlegende Fehlerarten
Excpetions können in den unterschiedlichsten Varianten und Situationen auftretten. Um etwas Ordnung in den Exception-Jungle zu bringen, ist es Sinnvoll die Excpetions in drei grobe Kategorien einzuteilen:
- **Nicht behebbare Excpetions**<br>
    Gewisse Fehler sind ausserhalb der Kontrolle des Entwicklers und können plötzlich und unerwartet auftretten. Bei solchen Exceptions ist ein Handling im Code oftmals nicht oder nur mit überproportionalem Aufwand möglich. Solche Fehler können nur durch robust entwickelte Systeme kompensiert werden (Systeme welche nach einem Fehler ihre Funktion selbstständig fortsetzen oder wiederaufnehmen können).

> *Beispiel:*
> - *Azure Function läuft in einen Execution Timeout / Überschreitet die verfügbare Computing Time*
> - *Arbeitsspeicher/Festplattenspeicher läuft während Programmlaufzeit voll.*
- **Durch retry behebare Excpetions**
> Beispiel: 
- **Durch aktive korrektur behebbare Excpetions**
> Beispiel: 

### Nicht behebbare Excpetions

In [66]:
#!powershell

# Running a Memoryhungry Function
Start-MemoryHungryFunction

Calculating Infinity...
InvalidOperation: 
Line |
   8 |          $PSCmdlet.ThrowTerminatingError($PSItem)
     |          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.


### Durch Retry behebare Excpetions
In dem nachfolgenden Beispiel wird Versucht auf einen Third-Party Server zuzugreifen. Da dieser Server teilweise von vielen Usern verwendet wird, ist es möglich, dass der Server zu busy ist um eine weitere Verbindun zuzulassen. In diesem Szennarion wird durch die Funktion `Connect-ThirdPartyServer` eine `System.ServiceModel.ServerTooBusyException` Exception geworfen.

Es ist Sinvoll mit einem zeitversetzten Retry auf diese Expetion zu reagieren und nur den Programfluss zu unterbrechen, wenn nach mehreren Versuchen immernoch keine Verbinung aufgebaut werden konnte. 

Im nachfolgenden Beispiel ist ein mögliches Exceptionhandling für diese Situation implementiert

> NOTE: Denk daran, es gibt immer verschiedene Möglichkeiten mit Exceptions umzugehen. Exceptionhandling ist eine anspruchsvolle Aufgabe für welche es kein Standartvorgehen gibt. Der einzig falsche Ansatz ist es, Exceptions zu Ignorieren 😉

In [69]:
#!powershell

# Connection Settings
$serverName = "server.test.work"
$retryTimeSeconds = 4

# Function used to Resolve ServerTooBusyExceptions
function Resolve-ServerConnectionBusy{
    <#
    .DESCRIPTION
        If the server is busy, the Scripts waits for the set ammount of seconds and then tries to connect again.
        If the second attempt failes too, an System.ServiceModel.ServerTooBusyException Exception is thrown.
    #>
    
    for ($i = $retryTimeSeconds; $i -gt 0; $i--) {
        Write-Information ("Retrying to connect to Server in {0} seconds" -f $i)
        Start-Sleep -Seconds 1
    }
    
    Connect-ThirdPartyServer $serverName
}

try{
    Connect-ThirdPartyServer $serverName
}
catch [System.ServiceModel.ServerTooBusyException]{
    Write-Error ("`n"+ $PSItem.ToString()) -Category ResourceBusy
    Resolve-ServerConnectionBusy

}

Write-Error: 
🔴 Server server.test.work is currently too busy to connect
Retrying to connect to Server in 4 seconds
Retrying to connect to Server in 3 seconds
Retrying to connect to Server in 2 seconds
Retrying to connect to Server in 1 seconds
🟢 Succesfully connected to Server server.test.work


### Durch aktive Korrektur behebbare Excpetions

In [ ]:
#!powershell

In [11]:
Describe 'Test' {
    It 'returns on text file'{
        Mock -CommandName Test-Path -MockWith {$false}
    }
}

Exception: /home/jovyan/.local/share/powershell/Modules/Pester/5.1.0/Pester.psm1:8589
Line |
8589 |          throw "Pester can run only saved files interactively. Please  …
     |          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Pester can run only saved files interactively. Please save your file to a disk.

